# **Project 2**
### Josh Winnes and Jon Contreras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

import geopandas as gpd
import shapely
import skmob
from skmob.measures.individual import jump_lengths
from skmob.tessellation import tilers
from skmob.preprocessing import detection, clustering
from skmob.preprocessing import filtering

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
import warnings
warnings.simplefilter("ignore")

import os
import sys

try:       
    kmz_file_path_josh = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Josh).kmz'
    kmz_file_path_jon = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data/March 7, 2024 - Whistler Blackcomb(Jon).kmz'
    gpx_file_name_josh = 'March 7, 2024 - Whistler Blackcomb(Josh).gpx'
    gpx_file_name_jon = 'March 7, 2024 - Whistler Blackcomb(Jon).gpx'
    script_directory = '/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/Math485/Project2'
    
    os.chdir('/Users/joshwinnes/Library/Mobile Documents/com~apple~CloudDocs/Wheaton College/fall 2024/topics in data science/data')
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    gpx_josh, run_info_josh = data_preprocessing(gpx_file_name_josh, kmz_file_path_josh)
    gpx_jon, run_info_jon = data_preprocessing(gpx_file_name_jon, kmz_file_path_jon)
    
    print('Working from Josh\'s laptop')
except FileNotFoundError:
    
    data_dir = 'your_data_directory_path'
    kmz_file_path_josh = 'path_to_josh_kmz_file'
    kmz_fil_path_jon = 'path_to_jon_kmz_file'
    gpx_file_name_josh = 'gpx_file_name_josh'
    gpx_file_name_jon = 'gpx_file_name_jon'
    script_directory = 'path_to_Project2'
    
    os.chdir(data_dir)
    sys.path.append(script_directory)
    from gpx_kmz_util import *
    
    gpx_josh, run_info_josh = data_preprocessing(gpx_file_name_josh, kmz_file_path_josh)
    gpx_jon, run_info_jon = data_preprocessing(gpx_file_name_jon, kmz_file_path_jon)
    
    print('Working from Jonathan\'s laptop')

Working from Josh's laptop


In [2]:
gpx_josh.head(30)

,Latitude,Longitude,Elevation,Time,Name
0,50.113260,-122.953950,683.549635,2024-03-07 09:46:36-08:00,pre_lift_1
1,50.113391,-122.953915,686.117031,2024-03-07 09:46:36-08:00,pre_lift_1
2,50.113319,-122.953772,688.737785,2024-03-07 09:48:27-08:00,pre_lift_1
3,50.113267,-122.953655,689.777724,2024-03-07 09:48:32-08:00,pre_lift_1
4,50.113181,-122.953594,692.476930,2024-03-07 09:49:04-08:00,pre_lift_1
5,50.113095,-122.953502,691.570411,2024-03-07 09:49:13-08:00,pre_lift_1
6,50.113067,-122.953366,691.382969,2024-03-07 09:49:35-08:00,Lift 1
7,50.112973,-122.953325,686.006277,2024-03-07 09:50:35-08:00,Lift 1
8,50.112877,-122.953391,681.919232,2024-03-07 09:51:15-08:00,Lift 1
9,50.112725,-122.953385,681.302449,2024-03-07 09:51:28-08:00,Lift 1


In [3]:
gpx_jon.head(30)

,Latitude,Longitude,Elevation,Time,Name
0,50.111385,-122.951516,699.522521,2024-03-07 09:52:51-08:00,pre_lift_1
1,50.109740,-122.953332,709.858664,2024-03-07 09:52:52-08:00,pre_lift_1
2,50.109765,-122.953274,741.954162,2024-03-07 09:52:53-08:00,pre_lift_1
3,50.109679,-122.953279,746.486242,2024-03-07 09:52:55-08:00,Lift 1
4,50.109586,-122.953275,748.005084,2024-03-07 09:52:58-08:00,Lift 1
5,50.109493,-122.953277,748.946129,2024-03-07 09:53:01-08:00,Lift 1
6,50.109389,-122.953282,749.281006,2024-03-07 09:53:03-08:00,Lift 1
7,50.109282,-122.953280,750.218117,2024-03-07 09:53:06-08:00,Lift 1
8,50.109176,-122.953279,751.754523,2024-03-07 09:53:09-08:00,Lift 1
9,50.109065,-122.953292,753.590171,2024-03-07 09:53:12-08:00,Lift 1


In [4]:
run_info_josh

,Name,Longitude,Latitude,Elevation,Time,time_elapsed,end_time
0,Lift 1,-122.953366,50.113067,691.382969,2024-03-07 09:49:35-08:00,0 days 00:22:04,2024-03-07 10:11:39-08:00
1,Run 1,-122.951056,50.078425,1681.088996,2024-03-07 10:11:39-08:00,0 days 00:10:54,2024-03-07 10:22:33-08:00
2,Lift 2,-122.963743,50.085441,1301.686260,2024-03-07 10:22:33-08:00,0 days 00:13:51,2024-03-07 10:36:24-08:00
3,Run 2,-122.947029,50.067096,1855.330032,2024-03-07 10:36:24-08:00,0 days 00:06:53,2024-03-07 10:43:17-08:00
5,Lift 3,-122.952236,50.066824,1779.142854,2024-03-07 10:43:17-08:00,0 days 00:04:03,2024-03-07 10:47:20-08:00
6,Run 3,-122.957573,50.059251,2183.242267,2024-03-07 10:47:20-08:00,0 days 01:17:11,2024-03-07 12:04:31-08:00
8,Lift 4,-122.964118,50.085837,1299.682279,2024-03-07 12:04:31-08:00,0 days 00:11:38,2024-03-07 12:16:09-08:00
9,Run 4,-122.947552,50.066370,1853.043952,2024-03-07 12:16:09-08:00,0 days 00:02:54,2024-03-07 12:19:03-08:00
10,Lift 5,-122.952248,50.066714,1772.088558,2024-03-07 12:19:03-08:00,0 days 00:03:50,2024-03-07 12:22:53-08:00
11,Run 5,-122.957416,50.059398,2178.784326,2024-03-07 12:22:53-08:00,0 days 00:15:21,2024-03-07 12:38:14-08:00


In [5]:
run_info_jon

,Name,Longitude,Latitude,Elevation,Time,time_elapsed,end_time
0,Lift 1,-122.953279,50.109679,746.486242,2024-03-07 09:52:55-08:00,0 days 00:17:35,2024-03-07 10:10:30-08:00
1,Run 1,-122.951039,50.079034,1678.810227,2024-03-07 10:10:30-08:00,0 days 00:12:03,2024-03-07 10:22:33-08:00
3,Lift 2,-122.963696,50.085477,1294.527448,2024-03-07 10:22:33-08:00,0 days 00:13:53,2024-03-07 10:36:26-08:00
4,Run 2,-122.947079,50.067145,1844.829283,2024-03-07 10:36:26-08:00,0 days 00:06:09,2024-03-07 10:42:35-08:00
5,Lift 3,-122.952012,50.067018,1772.808738,2024-03-07 10:42:35-08:00,0 days 00:04:37,2024-03-07 10:47:12-08:00
6,Run 3,-122.957425,50.059362,2170.953821,2024-03-07 10:47:12-08:00,0 days 01:04:53,2024-03-07 11:52:05-08:00
7,Lift 4,-122.988703,50.093790,656.509161,2024-03-07 11:52:05-08:00,0 days 00:39:09,2024-03-07 12:31:14-08:00
8,Run 4,-122.963043,50.058412,2059.593358,2024-03-07 12:31:14-08:00,0 days 01:45:49,2024-03-07 14:17:03-08:00
9,Lift 5 (Excelerator Express),-122.922710,50.111669,1126.007092,2024-03-07 14:17:03-08:00,0 days 00:09:30,2024-03-07 14:26:33-08:00
10,Run 5,-122.904938,50.103308,1634.285129,2024-03-07 14:26:33-08:00,0 days 00:16:58,2024-03-07 14:43:31-08:00


### ***Compiling Data into 1 DataFrame***

This is a change